In [5]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.estimator import SKLearn
import pandas as pd
from sklearn.datasets import load_diabetes

In [6]:
session = sagemaker.Session()
bucket = session.default_bucket()

[03/24/25 20:00:38] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=807434;file:///Users/zeynmehezmacbook/miniconda3/envs/deepenv/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=93665;file:///Users/zeynmehezmacbook/miniconda3/envs/deepenv/lib/python3.12/site-packages/botocore/credentials.py#1352\1352]8;;\

[03/24/25 20:00:43] INFO     Created S3 bucket: sagemaker-us-west-2-026090555467                     ]8;id=429967;file:///Users/zeynmehezmacbook/miniconda3/envs/deepenv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=378644;file:///Users/zeynmehezmacbook/miniconda3/envs/deepenv/lib/python3.12/site-packages/sagemaker/session.py#723\723]8;;\

In [9]:
get_execution_role()

[03/24/25 20:02:43] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=278445;file:///Users/zeynmehezmacbook/miniconda3/envs/deepenv/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=608217;file:///Users/zeynmehezmacbook/miniconda3/envs/deepenv/lib/python3.12/site-packages/botocore/credentials.py#1352\1352]8;;\

[03/24/25 20:02:47] WARNING  Couldn't call 'get_role' to get Role ARN from role name                ]8;id=794037;file:///Users/zeynmehezmacbook/miniconda3/envs/deepenv/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=184594;file:///Users/zeynmehezmacbook/miniconda3/envs/deepenv/lib/python3.12/site-packages/sagemaker/session.py#6019\6019]8;;\
                             tausif-developer to get Role path.                                                    

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 get_execution_role()                                                                         │
│   2                                                                                              │
│                                                                                                  │
│ /Users/zeynmehezmacbook/miniconda3/envs/deepenv/lib/python3.12/site-packages/sagemaker/session.p │
│ y:7985 in get_execution_role                                                                     │
│                                                                                                  │
│   7982 │   │   "The current AWS identity is not a role: {}, therefore it cannot be used as a "   │
│   7983 │   │   "SageMaker execution role"                                                        │
│   7984 │   )                                                                                     │
│ ❱ 7985 │   raise ValueError(message.format(arn))                                                 │
│   7986                                                                                           │
│   7987                                                                                           │
│   7988 def generate_default_sagemaker_bucket_name(boto_session):                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: The current AWS identity is not a role: arn:aws:iam::026090555467:user/tausif-developer, therefore it 
cannot be used as a SageMaker execution role

In [10]:
# Load simple dataset
diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = pd.DataFrame(diabetes.target, columns=['target'])
data = pd.concat([X, y], axis=1)

# Save to CSV and upload to S3
data.to_csv('diabetes.csv', index=False)
session = sagemaker.Session()
bucket = session.default_bucket()
prefix = 'sagemaker/diabetes'
input_s3 = session.upload_data('diabetes.csv', bucket=bucket, key_prefix=prefix)

# SageMaker role
role = 'arn:aws:iam::026090555467:role/SageMakerExecutionRole'

# Define a simple sklearn training script
train_script = '''
import pandas as pd
import joblib
from sklearn.linear_model import LinearRegression
import os

if __name__ == "__main__":
    input_data_path = "/opt/ml/input/data/train/diabetes.csv"
    data = pd.read_csv(input_data_path)
    X = data.drop(columns=['target'])
    y = data['target']

    model = LinearRegression()
    model.fit(X, y)

    model_dir = os.environ["SM_MODEL_DIR"]
    joblib.dump(model, f"{model_dir}/model.joblib")
'''

# Save the training script
with open("train.py", "w") as f:
    f.write(train_script)

# Set up and run the training job
sklearn_estimator = SKLearn(
    entry_point="train.py",
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version="0.23-1",
    py_version="py3",
    output_path=f"s3://{bucket}/{prefix}/output",
)

sklearn_estimator.fit({'train': input_s3})

# Deploy the trained model
predictor = sklearn_estimator.deploy(instance_type="ml.m5.large", initial_instance_count=1)


----------

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:55                                                                                   │
│                                                                                                  │
│   52 sklearn_estimator.fit({'train': input_s3})                                                  │
│   53                                                                                             │
│   54 # Deploy the trained model                                                                  │
│ ❱ 55 predictor = sklearn_estimator.deploy(instance_type="ml.m5.large", initial_instance_count    │
│   56                                                                                             │
│   57 # Test the endpoint in Python                                                               │
│   58 test_data = diabetes.data[0:1]                                                              │
│                                                                                                  │
│ /Users/zeynmehezmacbook/miniconda3/envs/deepenv/lib/python3.12/site-packages/sagemaker/estimator │
│ .py:1722 in deploy                                                                               │
│                                                                                                  │
│   1719 │   │   │   inference_tags=format_tags(tags), training_tags=self.tags                     │
│   1720 │   │   )                                                                                 │
│   1721 │   │                                                                                     │
│ ❱ 1722 │   │   return model.deploy(                                                              │
│   1723 │   │   │   instance_type=instance_type,                                                  │
│   1724 │   │   │   initial_instance_count=initial_instance_count,                                │
│   1725 │   │   │   serializer=serializer,                                                        │
│                                                                                                  │
│ /Users/zeynmehezmacbook/miniconda3/envs/deepenv/lib/python3.12/site-packages/sagemaker/model.py: │
│ 1786 in deploy                                                                                   │
│                                                                                                  │
│   1783 │   │   │   if is_explainer_enabled:                                                      │
│   1784 │   │   │   │   explainer_config_dict = explainer_config._to_request_dict()               │
│   1785 │   │   │                                                                                 │
│ ❱ 1786 │   │   │   self.sagemaker_session.endpoint_from_production_variants(                     │
│   1787 │   │   │   │   name=self.endpoint_name,                                                  │
│   1788 │   │   │   │   production_variants=[production_variant],                                 │
│   1789 │   │   │   │   tags=tags,                                                                │
│                                                                                                  │
│ /Users/zeynmehezmacbook/miniconda3/envs/deepenv/lib/python3.12/site-packages/sagemaker/session.p │
│ y:5940 in endpoint_from_production_variants                                                      │
│                                                                                                  │
│   5937 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   5938 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   5939 │   │                                                                                     │
│ ❱ 5940 │   │   return self.create_endpoint(                

In [11]:
predictor.delete_endpoint()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 predictor.delete_endpoint()                                                                  │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predictor' is not defined

In [ ]:
# Test the endpoint in Python
test_data = diabetes.data[0:1]
result = predictor.predict(test_data)
print(result)